In [2]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import openai
import os

openai.api_key = os.getenv('OPENAI_API_KEY')

# Financial Advisor Prompt

I'd like the assistant to have value investment baked in mind whenever it's exploring the answers.

In [46]:
FINANCIAL_ADVISOR_PROMPT = """\
You are a financial advisor in favor of value investment. You would give advice based on microeconomics, and whether the investment is an asset or liability.\
"""
FRIENDLY_WORDS_PROMPT = """\
You'll use friendly words for non-professionals audience. \
Sometimes, giving a few examples up to 2 is welcome." \
"""

BEING_CURIOUS_PROMPT_V1 = """\
Given a question, sort out underlying questions up to 3 and itemize them in bullets.\
"""

BEING_CURIOUS_PROMPT_V2 = """\
Given a question, come up with the underlying questions along with the short answer.

<< FORMATTING >>
Provide your answers as a JSON object with the following schema
{{
    "question": string \\ the question
    "answer": string \\ short answer generally covers the question
    "next_questions": ["", "", ...] \\ underlying questions
}}

REMEMBER: "question" MUST be the given question.
REMEMBER: "answer" should be short enough to fit in 1 or 2 sentence.
REMEMBER: "next_questions" are the underlying questions up to 10 or empy list if you don't find any.

REMEMBER: Don't provide advisory to the following topics for "answer", instead ask underlying questions for "next_questions":
- Whether to buy a house.
- Whether to buy specific stock, ETF, or bond.

<< QUESTION >>
{input}
"""

In [40]:
def ask(question: str) -> str:
  completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
          # System section
          
          {"role": "system", "content": FINANCIAL_ADVISOR_PROMPT},
          {"role": "system", "content": FRIENDLY_WORDS_PROMPT},
          
          # User section
          
          {
              "role": "user", 
              "content": BEING_CURIOUS_PROMPT_V2.format(input=question),
          },
      ],
      temperature=0,
  )
  
  return completion.choices[0].message.content

In [48]:
print(ask("Should I buy a house in Fremont, CA?"))

{
    "question": "Should I buy a house in Fremont, CA?",
    "answer": "As a financial advisor, I cannot provide specific advice on whether to buy a house in a particular location. However, I can help you consider the financial aspects of this decision.",
    "next_questions": [
        "What is your budget for buying a house?",
        "What are the current housing market conditions in Fremont, CA?",
        "What are your long-term financial goals?",
        "What are the potential risks and benefits of buying a house in Fremont, CA?"
    ]
}


In [47]:
print(ask("How many houses are currently available for sale in Fremont?"))

{
    "question": "How many houses are currently available for sale in Fremont?",
    "answer": "I'm sorry, but I don't have access to real-time data on the number of houses available for sale in Fremont.",
    "next_questions": []
}


In [18]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        # System section
        
        {"role": "system", "content": FINANCIAL_ADVISOR_PROMPT},
        {"role": "system", "content": BEING_CURIOUS_PROMPT_V1},
        {"role": "system", "content": FRIENDLY_WORDS_PROMPT},
        
        # Assistant section
        
        # TODO: Prompt user's current location
        # {"role": "assistant", "content": "For example:\nUser: Is it a good time to buy a house in Fremont?\nAnswer:\"},
        {"role": "assistant", "content": "Try to generate answer smaller than 120 words and use bullets to enhance the readability."},
        
#         {"role": "assistant", "content": 
# """
# Try to generate answer smaller than 120 words and use bullets to enhance the readability.

# For example:
# human: What's the difference of asset and liability?
# you: 
# - An asset is something that has value and can generate income or appreciate in value over time. Examples of assets include stocks, real estate, and businesses. 
# - A liability is something that represents a financial obligation or debt. Examples of liabilities include loans, mortgages, and credit card debt. 

# TL;DR, The main difference between assets and liabilities is that assets add value to your financial position, while liabilities subtract value.
# """,
# },
        
        # User section
        
        {"role": "user", "content": "Is it a good time to buy a house in Fremont, CA?"},
        # {"role": "user", "content": "What about buying a house in Truckee, CA for vacation house?"},
        # {"role": "user", "content": "Can you enumerate some cost of homeownership?"},
        # {"role": "user", "content": "What's the difference of asset and liability?"},
    ],
    temperature=0,
)

print(completion.choices[0].message.content)

To determine if it's a good time to buy a house in Fremont, CA, we need to consider a few factors:

1. Microeconomic factors: Is the housing market in Fremont currently experiencing growth or decline? Are there any local economic factors that could impact the housing market, such as job growth or population growth?

2. Value investment: Is the price of houses in Fremont considered undervalued or overvalued? Are there any potential opportunities for long-term appreciation in the housing market?

3. Asset or liability: Will purchasing a house in Fremont be an asset or a liability for you? Consider factors such as your financial situation, long-term goals, and the potential for rental income or property value appreciation.

It would be helpful to consult with a local real estate agent or financial advisor who specializes in the Fremont market to get a more accurate assessment of the current conditions and potential investment opportunities.


# Router

I'll use Langchain Router as backbone for Reasoner to  extract sub-queries and action

In [24]:
import langchain

from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

langchain.debug = True

physics_template = """\
You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}\
"""


math_template = """\
You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}\
"""

financial_advisor_template = """\
You are a financial advisor in favor of value investment. \
You would give advice based on microeconomics, and whether the investment is an asset or liability. \
You'll use friendly words for non-professionals audience. \
Sometimes, giving a few examples up to 2 is welcome. \

Here is a question:
{input}\
"""

echo_question_template = """\
You just copy and paste the question.

Here is a question:
{input}\
"""

prompt_infos = [
    # {
    #     "name": "physics",
    #     "description": "Good for answering questions about physics",
    #     "prompt_template": physics_template,
    # },
    # {
    #     "name": "math",
    #     "description": "Good for answering math questions",
    #     "prompt_template": math_template,
    # },
    
    {
        "name": "API",
        "description": "Good for looking up open API",
        "prompt_template": echo_question_template,
    },
    {
        "name": "finance",
        "description": "Good for answering financial questions",
        "prompt_template": financial_advisor_template,
    },
]

llm = OpenAI(openai_api_key=os.getenv('OPEN_AI_KEY'))
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt,)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt,)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

### Original Examples

In [35]:
print(chain.run("What is black body radiation?"))



> Entering new MultiPromptChain chain...


/opt/homebrew/anaconda3/envs/llm/lib/python3.11/site-packages/langchain/chains/llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


physics: {'input': 'What is black body radiation?'}
> Finished chain.


Black body radiation is the electromagnetic radiation emitted from a body due to its temperature. It is called "black body" because when all wavelengths of radiation are considered, the emission is considered to be like that of a black body. The spectrum of black body radiation is determined by Planck's law and is given by the Stefan-Boltzmann law.


In [37]:
print(
    chain.run(
        "What is the first prime number greater than 40 such that one plus the prime number is divisible by 3?"
    )
)



> Entering new MultiPromptChain chain...


/opt/homebrew/anaconda3/envs/llm/lib/python3.11/site-packages/langchain/chains/llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


math: {'input': 'What is the first prime number greater than 40 such that one plus the prime number is divisible by 3?'}
> Finished chain.


The first prime number greater than 40 such that one plus the prime number is divisible by 3 is 43. To arrive at this answer, we can start by listing out all the prime numbers greater than 40: 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97. From this list, we can see that the first prime number greater than 40 such that one plus the prime number is divisible by 3 is 43. To test this answer, we can add one to 43 to see if the sum is divisible by 3. 43 + 1 = 44, which is divisible by 3, thus verifying our answer.


### My Examples

In [ ]:
print(chain.run("What is black body radiation?"))

In [26]:
print(chain.run("What is QQQ's current price?"))

[chain/start] [1:chain:MultiPromptChain] Entering Chain run with input:
{
  "input": "What is QQQ's current price?"
}
[chain/start] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain] Entering Chain run with input:
{
  "input": "What is QQQ's current price?"
}
[chain/start] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain] Entering Chain run with input:
{
  "input": "What is QQQ's current price?"
}
[llm/start] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain > 4:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look

/opt/homebrew/anaconda3/envs/llm/lib/python3.11/site-packages/langchain/chains/llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain > 4:llm:OpenAI] [1.23s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{\n    \"destination\": \"finance\",\n    \"next_inputs\": \"What is QQQ's current price?\"\n}\n```",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "completion_tokens": 38,
      "total_tokens": 321,
      "prompt_tokens": 283
    },
    "model_name": "text-davinci-003"
  },
  "run": null
}
[chain/end] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain] [1.23s] Exiting Chain run with output:
{
  "text": "```json\n{\n    \"destination\": \"finance\",\n    \"next_inputs\": \"What is QQQ's current price?\"\n}\n```"
}
[chain/end] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain] [1.23s] Exiting Chain run with output:
{
  "destination": "finance",
  "next_inputs"

# Agent

In [30]:
import openai

from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI

openai.debug = False

llm = ChatOpenAI(model_name="gpt-4", temperature=0)
tools = load_tools(["ddg-search", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)

In [31]:
agent.run("Who directed the 2023 film Oppenheimer and what is their age? What is their age in days (assume 365 days per year)?")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Who directed the 2023 film Oppenheimer and what is their age? What is their age in days (assume 365 days per year)?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Who directed the 2023 film Oppenheimer and what is their age? What is their age in days (assume 365 days per year)?",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Answer the following questions as best you can. You have access to the following tools:\n\nduckduckgo_search: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: 

'The director of the 2023 film Oppenheimer is Christopher Nolan and he is 53 years old. In terms of days, he is approximately 19,345 days old.'

In [33]:
agent.run("What is QQQ's price today?")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is QQQ's price today?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "What is QQQ's price today?",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Answer the following questions as best you can. You have access to the following tools:\n\nduckduckgo_search: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [duckduckgo_search, Calculator]\nAction Input:

'The price of QQQ today is $364.70.'

In [34]:
agent.run("Analyze Meta company if it's worth of investment")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Analyze Meta company if it's worth of investment"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Analyze Meta company if it's worth of investment",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Answer the following questions as best you can. You have access to the following tools:\n\nduckduckgo_search: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [d

"Meta has strong financial performance and market presence, but also faces significant challenges and expenses. Therefore, whether or not to invest in Meta would depend on the investor's risk tolerance and investment strategy."

# OpenAPI

In [16]:
%%sh

curl -O https://raw.githubusercontent.com/openai/openai-openapi/master/openapi.yaml && \
mv openapi.yaml openai_openapi.yaml && \
curl -O https://www.klarna.com/us/shopping/public/openai/v0/api-docs && \
mv api-docs klarna_openapi.yaml && \
curl -O https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml && \
mv openapi.yaml spotify_openapi.yaml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  157k  100  157k    0     0  1153k      0 --:--:-- --:--:-- --:--:-- 1178k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2723    0  2723    0     0  16665      0 --:--:-- --:--:-- --:--:-- 17018
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  282k  100  282k    0     0   558k      0 --:--:-- --:--:-- --:--:--  561k


In [17]:
import os, yaml
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec
from langchain.llms.openai import OpenAI
from langchain.agents.agent_toolkits.openapi import planner
import spotipy.util as util
from langchain.requests import RequestsWrapper

with open("openai_openapi.yaml") as f:
    raw_openai_api_spec = yaml.load(f, Loader=yaml.Loader)
openai_api_spec = reduce_openapi_spec(raw_openai_api_spec)

with open("klarna_openapi.yaml") as f:
    raw_klarna_api_spec = yaml.load(f, Loader=yaml.Loader)
klarna_api_spec = reduce_openapi_spec(raw_klarna_api_spec)

with open("spotify_openapi.yaml") as f:
    raw_spotify_api_spec = yaml.load(f, Loader=yaml.Loader)
spotify_api_spec = reduce_openapi_spec(raw_spotify_api_spec)

requests_wrapper = RequestsWrapper(headers={})
llm = OpenAI(model_name="gpt-4", temperature=0.0)
spotify_agent = planner.create_openapi_agent(spotify_api_spec, requests_wrapper, llm)
user_query = (
    "make me a playlist with the first song from kind of blue. call it machine blues."
)
spotify_agent.run(user_query)

In [19]:
raw_openai_api_spec

{'openapi': '3.0.0',
 'info': {'title': 'OpenAI API',
  'description': 'The OpenAI REST API. Please see https://platform.openai.com/docs/api-reference for more details.',
  'version': '2.0.0',
  'termsOfService': 'https://openai.com/policies/terms-of-use',
  'contact': {'name': 'OpenAI Support', 'url': 'https://help.openai.com/'},
  'license': {'name': 'MIT',
   'url': 'https://github.com/openai/openai-openapi/blob/master/LICENSE'}},
 'servers': [{'url': 'https://api.openai.com/v1'}],
 'tags': [{'name': 'OpenAI', 'description': 'The OpenAI REST API'}],
 'paths': {'/chat/completions': {'post': {'operationId': 'createChatCompletion',
    'tags': ['OpenAI'],
    'summary': 'Creates a model response for the given chat conversation.',
    'requestBody': {'required': True,
     'content': {'application/json': {'schema': {'$ref': '#/components/schemas/CreateChatCompletionRequest'}}}},
    'responses': {'200': {'description': 'OK',
      'content': {'application/json': {'schema': {'$ref': '#/c

In [ ]:
openai_api_spec